### 数据统计与分析

In [1]:
import re 
import numpy as np
import pandas as pd
import jieba 

from wordcloud import WordCloud

In [2]:
file_path = './data.csv'
data = pd.read_csv(file_path)

> 标题的词云

In [3]:
TITLE_FILE_NAME = './temp/titles.txt'

fp_title = open(TITLE_FILE_NAME, 'w+' , encoding='utf-8')
titles = data['标题']
for title in titles:
  fp_title.write(title[title.find('（'):] if title.find('（') != -1 else ' ')

# 把标题写入文件
fp_title.close()

# 修正标题
data['标题'] = data['标题'].apply(lambda x: x[:x.find('（')] if x.find('（') != -1 else x)

In [4]:
# 注意到获取的页面内容中有html的标签和其他符号存在，不利于下一步的词云操作，所以需要利用jieba和正则表达式对得到的文本进行清洗
r ='[，。\%、；1234567890n-】【“”]《》（）'

file=open(TITLE_FILE_NAME, "r", encoding='utf-8').read()

# 剔除无关信息
file =re.sub(r,'',file)
# 删除中文无关词汇
chinese_char = '[的版]'
file = re.sub(chinese_char,'',file)

#分词
con = jieba.lcut(file)

#分词后插入空格
words = " ".join(con)

#词云分析
wordcloud = WordCloud(font_path="./assets/1570766042.ttf",background_color="white",width=1300, height=800).generate(words)

wordcloud.to_file('./assets/title_cloud.png') 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.216 seconds.
Prefix dict has been built successfully.


<!-- ![](./assets/title_cloud.png) -->

从词云图中可以看出消费者更加倾向于购买标题中含有“全册”、“推荐”、“代表作”等词的书籍


> 评论数量

In [14]:
# 排序后的DataFrame
data_sort_by_comment = data.sort_values('评论数量', ascending=False)

# 评论数量最多的 top10、30
# data_sort_by_comment.head(10)
# data_sort_by_comment.head(30)

,排名,标题,评论数量,推荐率,出版社,作者,出版时间,价格,类别1,类别2
18,19,东野圭吾：解忧杂货店,2466104,99.70%,南海出版公司,（日）东野圭吾 著， 新经典 出品,2020,5,图书,小说
127,128,人间失格,2196134,99.80%,作家出版社,（日）太宰治 著，杨伟 译,2015,1,小说,社会小说
7,8,马尔克斯：百年孤独,2061178,99.80%,南海出版公司,加西亚·马尔克斯 著，新经典 出品,2017,3,小说,世界名著
26,27,东野圭吾：白夜行,1936755,99.90%,南海出版公司,[日]东野圭吾 著，新经典 出品,2017,5,小说,社会小说
40,41,正面管教(修订版),1920551,100%,北京联合出版公司,[美] 简·尼尔森,2016,3,亲子/家教,家教方法
8,9,三体：全三册 刘慈欣代表作，亚洲“雨果奖”获奖作品！,1789154,99.80%,重庆出版社,刘慈欣,2010,5,小说,科幻小说图书
64,65,神奇校车·图画书版,1677734,99.90%,贵州人民出版社,"(美)乔安娜柯尔 著 ,(美)布鲁斯·迪根 图",2018,10,童书,科普/百科
91,92,窗边的小豆豆(2018版),1537547,99.90%,南海出版公司,（日）黑柳彻子， 爱心树童书 出品,2018,2,童书,外国儿童文学
162,163,我们仨,1533940,99.80%,生活.读书.新知三联书店,杨绛,2018,2,传记,文学家
24,25,小熊和最好的爸爸,1497197,99.60%,贵州人民出版社,（荷）阿兰德·丹姆 著，（荷）亚历克斯·沃尔夫 绘，漆仰平，爱桐 译,2007,3,童书,绘本/图画书


> top10出版社以及上榜的书籍

In [6]:
data.head(5)

,排名,标题,评论数量,推荐率,出版社,作者,出版时间,价格,类别1,类别2
0,1,人生海海,894784,100%,北京十月文艺出版社,麦家 著 ，新经典 出品,2019/4/16,¥55.00,小说,中国当代小说
1,2,大话中国艺术史,41862,99.90%,海南出版社,意公子；读客文化 出品,2022/3/1,¥112.10,艺术,世界各国艺术概况
2,3,生死疲劳,209949,100%,浙江文艺出版社,莫言；读客文化 出品,2022/1/1,¥66.40,小说,中国当代小说
3,4,蛤蟆先生去看心理医生,1077140,100%,天津人民出版社,罗伯特·戴博德，果麦文化 出品,2020/7/1,¥28.50,心理学,心灵疗愈
4,5,被讨厌的勇气：“自我启发之父”阿德勒的哲学课 岸见一郎,603965,100%,机械工业出版社,"岸見一郎, 古賀史健",2020/3/5,¥30.00,成功/励志,心灵与修养


In [7]:
data_groupby_publisher = data.groupby(['出版社'])

# 将 DataFrameGroupBy 转换为 array 类型
# 然后再每一个group（出版社）中添加这个出版社一共出版的图书
# 然后把这个array按照出版图书的降序排序

publisher_with_rank = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_publisher)], dtype=object)])
publisher_with_rank = publisher_with_rank[publisher_with_rank[:,0].argsort()][::-1]

# 所有出版社按照上榜图书排序和他们的图书
# publisher_with_rank

> 一类图书类别统计

In [8]:
data_groupby_type_1 = data.groupby(['类别1'])
type_1_book_group = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_type_1)], dtype=object)])
type_1_book_group = type_1_book_group[type_1_book_group[:,0].argsort()][::-1]

# 所有1级分类和分类下面的图书
# type_1_book_group

> 二类图书类别统计

In [9]:
data_groupby_type_2 = data.groupby(['类别2'])

type_2_book_group = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_type_2)], dtype=object)])
type_2_book_group = type_2_book_group[type_2_book_group[:,0].argsort()][::-1]

# 所有2级分类和分类下面的图书
# type_2_book_group

> 热销图书的定价区间，大家最喜欢什么样的价位的图书

In [10]:
# 价格区间以10为step
data_modify_price = data
data_modify_price = data_modify_price['价格'].apply(lambda x: int(float(x[1:]) / 10))
data_with_price_df = data
# 把价格修正为价格区间
data_with_price_df['价格'] = data_modify_price

In [11]:
# 按照价格分组
data_groupby_price = data_with_price_df.groupby(['价格'])
price_book_group = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_price)], dtype=object)])
price_book_group = price_book_group[price_book_group[:,0].argsort()][::-1]

# 所有的价格区间和区间内的图书数量和内容
# price_book_group

> 出版年份和时间排序，大家最喜欢什么时候出版的书

In [12]:
# 年份以年排序
data_modify_date = data
data_modify_date = data_modify_date['出版时间'].apply(lambda x: x[:4])
data_with_date_df = data
# 把价格修正为价格区间
data_with_date_df['出版时间'] = data_modify_date

In [13]:
# 按照价格分组
data_groupby_date = data_with_date_df.groupby(['出版时间'])
date_book_group = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_date)], dtype=object)])
date_book_group = date_book_group[date_book_group[:,0].argsort()][::-1]

# 所有的价格区间和区间内的图书数量和内容
# date_book_group

### 数据可视化